# Sub Module 1 - Text extraction

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
!pip install pdfminer.six
!pip install pdfplumber
!pip install mammoth
!pip install pytesseract

Defaulting to user installation because normal site-packages is not writeable
  Using cached pdfminer_six-20250327-py3-none-any.whl (5.6 MB)
Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 60.2/60.2 kB 458.7 kB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 2.3 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 52.9/52.9 kB 341.3 kB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!pip install pytesseract pillow

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pytesseract
from PIL import Image

# Point pytesseract to your Tesseract installation
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Test with a simple image
try:
    print(pytesseract.get_tesseract_version())
    print("Tesseract installed successfully!")
except:
    print("Installation failed - check your Tesseract path")

5.5.0.20241111
Tesseract installed successfully!


In [5]:
from PIL import Image
import pytesseract

# Set the path to tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Example usage
def extract_text_from_image(image_path):
    """Extract text from an image file"""
    try:
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

# Test with a resume image
resume_text = extract_text_from_image('img-resume.png')
if resume_text:
    print(resume_text[:500])  # Print first 500 characters

Devika Patel erseyrer)

Secretary bes
Work Experience
Self-Employed  Stay-At-Home Mom

+ Managed busget fora family ofS saving over $3500 per year

+ Sueamtined nousehols chore processes, increasing eficency and productivity by 52%

+ Coordinated S+ monty appointments & engagements wih 97% accuracy

1 Taught 3 cldren ages 46, and 1) how to cookheathy meals according persona
preferences and ned guidelines

US National Park Service - Secretary (0A)
+ Managed schedules for 8 colesgues booking appoi


In [6]:
import pdfplumber
from pdfminer.high_level import extract_text
def extract_text_from_pdfs(pdf_path):
    # First pass: use pdfplumber to check for tables
    has_tables = False

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            if tables and any(any(row) for row in tables):
                has_tables = True
                break

    # If tables are detected, extract with pdfplumber
    if has_tables:
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text.strip() + "\n"
        return text.strip()

    # Otherwise, extract with PyMuPDF (better for layouts without tables)
    else:
        return extract_text(pdf_path)


In [7]:
import mammoth

def extract_text_with_mammoth(docx_path):
    """Extract text from docx file using mammoth."""
    with open(docx_path, "rb") as docx_file:
        result = mammoth.extract_raw_text(docx_file)
        return result.value

In [8]:
def extract_clean_text_from_txt(txt_path: str) -> str:
    """Extracts and cleans text (removes excess whitespace, preserves sections)."""
    with open(txt_path, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines() if line.strip()]

    # Join lines while preserving paragraph breaks (double newlines)
    text = "\n".join(lines)
    # Collapse 3+ newlines into 2
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text

In [9]:
from PIL import Image
import pytesseract

def extract_text_from_image(image_path):
    """Extract text specifically from image files with more control"""
    try:
        # Open the image
        img = Image.open(image_path)

        # Use pytesseract to extract text
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Error extracting text from image {image_path}: {e}")
        return ""

In [10]:
import re
from typing import Union
import os

def extract_text_from_resume(file_path: str) -> Union[str, None]:
    """
    Unified function to extract text from resumes in various formats (PDF, DOCX, TXT, Images).
    Automatically detects file type and uses the appropriate extraction method.

    Args:
        file_path: Path to the resume file

    Returns:
        Extracted text as string, or None if extraction fails/unsupported format
    """
    # Validate file exists
    if not os.path.exists(file_path):
        return f"Error: File not found at {file_path}"

    # Get file extension
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()

    try:
        # PDF handling
        if ext == '.pdf':
            return extract_text_from_pdfs(file_path)

        # DOCX handling
        elif ext == '.docx':
            return extract_text_with_mammoth(file_path)


        # TXT handling
        elif ext == '.txt':
            return extract_clean_text_from_txt(file_path)

        # Image handling (JPEG, JPG, PNG)
        elif ext in ('.jpg', '.jpeg', '.png'):
            return extract_text_from_image(file_path)

        # Unsupported format
        else:
            supported = ['.pdf', '.docx', '.txt', '.jpg', '.jpeg', '.png']
            return f"Error: Unsupported file type {ext}. Supported formats: {', '.join(supported)}"

    except Exception as e:
        return f"Error extracting text from {file_path}: {str(e)}"



In [11]:
import re

def remove_empty_lines_regex(text):
    """
    Remove empty lines using regular expressions.

    Args:
        text (str): Input text

    Returns:
        str: Text with empty lines removed
    """
    return re.sub(r'\n\s*\n', '\n', text).strip()

In [12]:
raw_cv_text = extract_text_from_resume('Yusin-Resume.pdf')
print(raw_cv_text)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


YUSIN ALI ADAN 

yunisaden3@gmail.com • 0798654423 • Linkedin.com • Github.com 

EDUCATION 

Jomo Kenyatta University of Agriculture and Technology | Juja, Kiambu                                                                                   

Bachelor of Science in Computer Science                                                        

September 2021- December 2025 

EXPERIENCE  

JHUB Africa | Juja, Kiambu   

Software Engineer Intern 

            May 2024 - Sep 2024 

-Led backend development for Kenya's Digital Ecosystem mapping platform, implementing RESTful APIs with Node.js 

and Express.js 

-Engineered microservices architecture for processing and analyzing startup ecosystem data 

-Collaborated with stakeholders to gather requirements and implement technical solutions 

Technologies: Node.js, PostgreSQL, Docker, Git, REST APIs 

TECHNICAL PROJECTS 

SafariTech | Personal Project   
-Built a full-stack platform connecting startup founders and students to facilitate mento

In [14]:
raw_cv_text = extract_text_from_resume('Engineering_Industrial_Tech_resume.docx')
print(raw_cv_text)



SUMMARY OF QUALIFICATIONS

Experienced in designing solutions for environmental problems that require integration of environmental and human health constraints compatible with economic growth, sustainable development, and ethical standards

Proficient with a variety of modeling and drafting software including AutoCAD and ALGOR

Efficiently manage projects and collaborate with teams to accelerate the overarching goals of the company





EDUCATION

Southeastern Louisiana University – Hammond, LA                                                                                                                May 2024

Bachelor of Science in Engineering Technology

Concentration: Electrical Energy Engineering

Cumulative GPA: 3.75/4.00

Relevant Coursework: Introduction to Programming, Electrical Circuits, Electromagnetics, Programming for Technologists





TECHNICAL PROFICIENCIES

Modeling Programs: ALGOR, eQUEST, and EnergyPro (LEED project with a VRF system)

Drafting Software: AutoCAD

# Sub Module 2 - Job Category/Designation Extractor

In [15]:
import pandas as pd

# Read the Excel file into a DataFrame
df = pd.read_excel('job_roles_database.xlsx')  # Replace with your file path

# Verify the column exists
if 'Job_Role' not in df.columns:
    raise ValueError("The DataFrame doesn't contain a 'Job_Role' column")

# Extract unique job roles and convert to list
all_job_roles = df['Job_Role'].unique().tolist()

# Display the first 10 roles as a sanity check
print(f"Found {len(all_job_roles)} unique job roles. First 10:")
print(all_job_roles[:10])

Found 1455 unique job roles. First 10:
['Political Scientists', 'Municipal', 'Office Machine Operators', 'Receptionists', 'Paralegals', 'Teacher', 'Radio Disc Jockeys', 'Equal Opportunity Representatives', 'Preventive Medicine Physicians', 'Aircraft Structure']


In [16]:
def role_exists(role, all_job_roles):
    return role.lower() in [r.lower() for r in all_job_roles]


In [25]:
def remove_job_roles(all_job_roles, items_to_remove):
    """
    Remove specified items from the job roles list if they exist
    
    Args:
        all_job_roles (list): Original list of job roles
        items_to_remove (list): Items to remove from the list
        
    Returns:
        list: Filtered job roles list
    """
    # Convert both lists to lowercase sets for case-insensitive comparison
    original_set = {role.lower() for role in all_job_roles}
    remove_set = {item.lower() for item in items_to_remove}
    
    # Remove items that exist in both sets
    filtered_roles = [role for role in all_job_roles 
                     if role.lower() not in remove_set]
    
    # Report what was removed
    removed_items = original_set.intersection(remove_set)
    if removed_items:
        print(f"Removed {len(removed_items)} items: {', '.join(removed_items)}")
    else:
        print("No matching items found to remove")
    
    return filtered_roles

In [26]:
all_job_roles = remove_job_roles(all_job_roles, ['Software', 'Computer', 'Data', 'IT', 'pressing', 'Hardware'])

Removed 2 items: pressing, computer


In [27]:
def add_unique_job_roles(job_roles, all_job_roles):
    existing_roles_lower = {role.lower() for role in all_job_roles}
    for role in job_roles:
        if role.lower() not in existing_roles_lower:
            all_job_roles.append(role)
            existing_roles_lower.add(role.lower())  # update the set to reflect the new addition


In [29]:
add_unique_job_roles(['UI/UX Designer', 'Devops', 'Developer', 'Data Analyst', 'Designer'], all_job_roles)

In [34]:
!python -m pip install --upgrade pip setuptools wheel


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.8/1.8 MB 1.9 MB/s eta 0:00:00
  Using cached setuptools-78.1.0-py3-none-any.whl (1.3 MB)
  Using cached wheel-0.45.1-py3-none-any.whl (72 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


In [35]:
!pip install spacy --prefer-binary

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 2.5/2.5 MB 2.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 12.3/12.3 MB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 2.0/2.0 MB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 633.4/633.4 kB 2.4 MB/s eta 0:00:00
   --------

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [36 lines of output]
  BLIS_COMPILER? None
  C:\Users\Ashraf Mohammed\AppData\Local\Temp\pip-build-env-uqeoi77o\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!
  
          ********************************************************************************
          Please consider removing the following classifiers in favor of a SPDX license expression:
  
          License :: OSI Approved :: BSD License
  
          See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
          ********************************************************************************
  
  !!
    self._finalize_license_expression()
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-39\blis
  copying blis\about.py -> build\lib.win-amd64-cpython-39\blis
  copying blis\benchmark.py -> build\lib.win-a

In [32]:
import spacy
from spacy.matcher import PhraseMatcher
from collections import defaultdict

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_job_roles(text, all_job_roles):
    """
    Extract the most frequent job role from text using exact matching.
    Returns the most frequent role, or first one if tied, or None if none found.
    """
    # Create PhraseMatcher and patterns
    job_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    job_patterns = [nlp.make_doc(role) for role in all_job_roles]
    job_matcher.add("JOB_ROLE", job_patterns)
    
    doc = nlp(text)
    matches = job_matcher(doc)
    
    # Count frequency of each matched role
    role_counts = defaultdict(int)
    occurrence_order = {}  # To track first occurrence position
    
    for match_id, start, end in matches:
        role = doc[start:end].text.strip()
        role_counts[role] += 1
        if role not in occurrence_order:
            occurrence_order[role] = len(occurrence_order)
    
    if not role_counts:
        return None
    
    # Find role with highest count
    max_count = max(role_counts.values())
    candidates = [role for role, count in role_counts.items() if count == max_count]
    
    # If tie, return the one that appeared first in the text
    if len(candidates) > 1:
        # Sort by first occurrence position
        candidates.sort(key=lambda x: occurrence_order[x])
    
    return candidates[0]

def predict_job_role(resume_text, all_job_roles, model_components):
    """
    Combined prediction pipeline:
    1. First try exact matching with extract_job_roles
    2. If that returns None, use the trained model
    """
    # First try exact matching
    exact_role = extract_job_roles(resume_text, all_job_roles)
    
    if exact_role is not None:
        return exact_role
    
    # If no exact match found, use the trained model
    cleaned_resume = model_components['clean_resume'](resume_text)
    resume_tfidf = model_components['tfidf'].transform([cleaned_resume])
    predicted_category_encoded = model_components['model'].predict(resume_tfidf)
    predicted_role = model_components['label_encoder'].inverse_transform(predicted_category_encoded)
    
    return predicted_role[0]

# # Example usage:
# # Load your saved model components
# with open('resume_classifier_tuned.pkl', 'rb') as f:
#     model_components = pickle.load(f)



# Example resume
resume_text = """
John Doe
Data Scientist
Experience:
- Built machine learning models for customer segmentation
- Developed NLP pipelines for text classification
- Strong skills in Python, TensorFlow, and SQL
Education:
MS in Computer Science, University X
"""

job_role = extract_job_roles(resume_text, all_job_roles)
print(job_role)

ModuleNotFoundError: No module named 'spacy'